# Prompt, Mask, Pipeline-Predict

# Notebook:

##### Imports
##### replace 'MajorClaim' by 'Stance'
##### make train and test datasets
##### train BERT Mask on the train dataset
##### save the model
##### create a pipeline using the saved model
##### run predictions on it
##### evaluate with the test set

### Imports

In [1]:
!pip install transformers
!pip install ipywidgets
!pip install IProgress
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 4.7 MB 15.5 MB/s eta 0:00:01
     |████████████████████████████████| 6.6 MB 43.8 MB/s eta 0:00:01
     |████████████████████████████████| 101 kB 55.9 MB/s ta 0:00:01
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 133 kB 13.9 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 43.9 MB/s eta 0:00:01
     |████████████████████████████████| 383 kB 43.5 MB/s eta 0:00:01
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 11.7 MB 13.6 MB/s eta 0:00:01


In [2]:
import transformers
import torch

In [3]:
from transformers import BertTokenizer, BertForMaskedLM

In [4]:
import pandas as pd

### tokenizer

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

### make the train, test dataframe

In [6]:
df = pd.read_pickle("/notebooks/Prompting/dataset/pe_dataset_w_prompts_1_pickle")

In [7]:
# rename MajorClaim as stance

In [8]:
df

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,nr_preceeding_comps_in_para,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_1
0,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.25,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, VERB, VERB, ADJ, NO...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
1,essay001,T2,MajorClaim 2154 2231,a more cooperative attitudes towards life is m...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,1.00,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADV, ADJ, NOUN, ADP,...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
2,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,0,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
3,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,1,2,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
4,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,2,1,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5968,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,4,3,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying har...
5969,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,5,2,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, VERB, NOUN, VE...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying har...
5970,essay402,T13,Premise 1393 1436,they can contribute positively to community,Premise,Premise,[],Support,Linked,TRAIN,...,6,1,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: PRON, VERB, VERB, ADV, AD...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying ha

In [9]:
def rename_MC(x):
    
    prompted_rep = x.prompted_representation_1
    
    last_token = prompted_rep.split()[-1]
    
    if last_token == 'MajorClaim.':
    
        new_rep = prompted_rep.replace(last_token, "Stance.")
        
    else:
        
        new_rep = prompted_rep
    
    return new_rep
    
    

In [10]:
rename_MC(df.iloc[0])

'Component: Topic: Should students be taught to compete or to cooperate?, Sentence: From this point of view, I firmly believe that we should attach more importance to cooperation during primary education., First or last in essay: Yes, First in paragraph: Yes, Last in paragraph: Yes, In in introduction: Yes, Is in conclusion: No. Part Of Speech tags: PRON, VERB, VERB, ADJ, NOUN, ADP, NOUN, ADP, ADJ, NOUN. Is this component a premise, a claim or a major claim? This component is a Stance.'

In [11]:
# correct now. now run it on the whole df.

In [12]:
df['prompted_rep_w_modif_MC'] = df.apply(lambda x: rename_MC(x), axis=1)

In [13]:
df['prompted_rep_w_modif_MC'][0]

'Component: Topic: Should students be taught to compete or to cooperate?, Sentence: From this point of view, I firmly believe that we should attach more importance to cooperation during primary education., First or last in essay: Yes, First in paragraph: Yes, Last in paragraph: Yes, In in introduction: Yes, Is in conclusion: No. Part Of Speech tags: PRON, VERB, VERB, ADJ, NOUN, ADP, NOUN, ADP, ADJ, NOUN. Is this component a premise, a claim or a major claim? This component is a Stance.'

In [ ]:
# change the effing thing 'major claim' --> stance

In [14]:
# now do a sanity check to see if it's correct

In [15]:
last_tokens_list = []

In [16]:
def get_last_token(x):
    
    prompted_rep = x.prompted_rep_w_modif_MC
    
    last_token = prompted_rep.split()[-1]
    
    last_tokens_list.append(last_token)

In [17]:
df.apply(lambda x: get_last_token(x), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
5968    None
5969    None
5970    None
5971    None
5972    None
Length: 5973, dtype: object

In [18]:
last_tokens_list

['Stance.',
 'Stance.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Stance.',
 'Stance.',
 'Premise.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Stance.',
 'Stance.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Stance.',
 'Stance.',
 'Claim.',
 'Claim.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Stance.',
 'Stance.',
 'Claim.',
 'Claim.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Stance.',
 'Stance.',
 'Claim.',
 'Claim.',
 'Claim.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Stance.',
 'Claim.',
 'Premise.',
 'Premise.',


In [19]:
set(last_tokens_list)

{'Claim.', 'Premise.', 'Stance.'}

In [20]:
####### YAAAAAAAAASSSSSSSSSS KINGGGGGGGGGGGGGGGG

## make train, test datasets

### train dataset

In [21]:
df_train = df[df.split == 'TRAIN']

In [22]:
df_train = df_train.reset_index(drop=True)

In [23]:
df_train

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_1,prompted_rep_w_modif_MC
0,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.25,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, VERB, VERB, ADJ, NO...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...,Component: Topic: Should students be taught to...
1,essay001,T2,MajorClaim 2154 2231,a more cooperative attitudes towards life is m...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,1.00,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADV, ADJ, NOUN, ADP,...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...,Component: Topic: Should students be taught to...
2,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...,Component: Topic: Should students be taught to...
3,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,2,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...,Component: Topic: Should students be taught to...
4,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,1,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...,Component: Topic: Should students be taught to...,Component: Topic: Should students be taught to...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4708,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,3,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying har...,Component: Topic: Children should studying har...
4709,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,2,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, VERB, NOUN, VE...",Topic: Children should studying hard or playin...,Component: Topic: Children should studying har...,Component: Topic: Children should studying har...
4710,essay402,T13,Premise 1393 1436,they can contribute positively to community,Premise,Premise,[],S

In [24]:
df_train.split.value_counts()

TRAIN    4713
Name: split, dtype: int64

In [25]:
prompted_texts_train = df_train['prompted_rep_w_modif_MC'][:].tolist()

In [26]:
len(prompted_texts_train)

4713

In [27]:
inputs_train = tokenizer(prompted_texts_train, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [28]:
inputs_train

{'input_ids': tensor([[ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        ...,
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [29]:
inputs_train['input_ids'].shape

torch.Size([4713, 512])

In [30]:
# for training we will need labels

In [31]:
inputs_train['labels'] = inputs_train.input_ids.detach().clone()

In [32]:
inputs_train

{'input_ids': tensor([[ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        ...,
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        ...,
      

In [33]:
inputs_train['labels'].shape

torch.Size([4713, 512])

In [34]:
# find where the last 102 is

In [35]:
(inputs_train['input_ids'][0] == 102).nonzero(as_tuple=True)[0].item()

121

In [36]:
tokenizer.decode(4366)

'c l a i m'

In [37]:
tokenizer.decode(18458)

'p r e m i s e'

In [38]:
# claim = 4366, premise = 18458

In [39]:
(inputs_train['input_ids'][1] == 102).nonzero(as_tuple=True)[0].item()

146

In [40]:
inputs_train['input_ids'][1][146]

tensor(102)

In [41]:
# confirm that all at [len-2] indices are either claim or premise

In [42]:
sep_indices_list = []

for i in range(len(prompted_texts_train)):
    sep_idx = (inputs_train['input_ids'][i] == 102).nonzero(as_tuple=True)[0].item()
    sep_indices_list.append(sep_idx)

In [43]:
sep_indices_list

[121,
 146,
 145,
 171,
 220,
 150,
 101,
 130,
 152,
 288,
 114,
 140,
 154,
 143,
 113,
 188,
 139,
 149,
 124,
 149,
 119,
 129,
 176,
 142,
 135,
 162,
 143,
 105,
 140,
 179,
 151,
 131,
 119,
 208,
 141,
 128,
 125,
 131,
 221,
 207,
 161,
 153,
 136,
 119,
 124,
 157,
 127,
 132,
 144,
 136,
 198,
 180,
 170,
 148,
 170,
 143,
 170,
 165,
 127,
 104,
 121,
 107,
 110,
 92,
 111,
 133,
 155,
 109,
 131,
 100,
 124,
 98,
 128,
 131,
 156,
 102,
 126,
 140,
 138,
 128,
 177,
 112,
 139,
 151,
 152,
 117,
 129,
 133,
 143,
 115,
 126,
 105,
 113,
 116,
 148,
 157,
 163,
 117,
 124,
 99,
 161,
 116,
 130,
 160,
 130,
 118,
 117,
 102,
 115,
 109,
 126,
 96,
 130,
 231,
 215,
 102,
 120,
 146,
 110,
 127,
 145,
 144,
 170,
 150,
 119,
 123,
 116,
 119,
 102,
 107,
 142,
 101,
 95,
 186,
 133,
 154,
 130,
 132,
 134,
 124,
 128,
 150,
 95,
 153,
 123,
 139,
 122,
 127,
 149,
 105,
 149,
 116,
 119,
 155,
 88,
 104,
 98,
 105,
 147,
 143,
 112,
 165,
 146,
 151,
 150,
 116,
 196,
 137,


In [44]:
len(sep_indices_list)

4713

In [45]:
class_tokens_indices_list = [x - 2 for x in sep_indices_list]

In [46]:
class_tokens_indices_list

[119,
 144,
 143,
 169,
 218,
 148,
 99,
 128,
 150,
 286,
 112,
 138,
 152,
 141,
 111,
 186,
 137,
 147,
 122,
 147,
 117,
 127,
 174,
 140,
 133,
 160,
 141,
 103,
 138,
 177,
 149,
 129,
 117,
 206,
 139,
 126,
 123,
 129,
 219,
 205,
 159,
 151,
 134,
 117,
 122,
 155,
 125,
 130,
 142,
 134,
 196,
 178,
 168,
 146,
 168,
 141,
 168,
 163,
 125,
 102,
 119,
 105,
 108,
 90,
 109,
 131,
 153,
 107,
 129,
 98,
 122,
 96,
 126,
 129,
 154,
 100,
 124,
 138,
 136,
 126,
 175,
 110,
 137,
 149,
 150,
 115,
 127,
 131,
 141,
 113,
 124,
 103,
 111,
 114,
 146,
 155,
 161,
 115,
 122,
 97,
 159,
 114,
 128,
 158,
 128,
 116,
 115,
 100,
 113,
 107,
 124,
 94,
 128,
 229,
 213,
 100,
 118,
 144,
 108,
 125,
 143,
 142,
 168,
 148,
 117,
 121,
 114,
 117,
 100,
 105,
 140,
 99,
 93,
 184,
 131,
 152,
 128,
 130,
 132,
 122,
 126,
 148,
 93,
 151,
 121,
 137,
 120,
 125,
 147,
 103,
 147,
 114,
 117,
 153,
 86,
 102,
 96,
 103,
 145,
 141,
 110,
 163,
 144,
 149,
 148,
 114,
 194,
 135,
 10

In [47]:
# inputs_pe['input_ids'][0][143].item() # correct

In [48]:
# now get a list of input ids at all these indics: they should be either claim = 4366, premise = 18458

In [49]:
list_index_minus_2 = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_2 = inputs_train['input_ids'][idx][val].item()
    list_index_minus_2.append(at_idx_minus_2)

In [50]:
set(list_index_minus_2)

{4366, 11032, 18458}

In [51]:
tokenizer.decode(11032)

's t a n c e'

In [52]:
# correct

In [53]:
# now make the class token indices 103

In [54]:
for idx, val in enumerate(class_tokens_indices_list):
    inputs_train['input_ids'][idx][val] = 103

In [55]:
# now check if they are 103

In [56]:
unsontroi_check_list = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_2 = inputs_train['input_ids'][idx][val].item()
    unsontroi_check_list.append(at_idx_minus_2)

In [57]:
set(unsontroi_check_list)

{103}

In [58]:
# correct

In [59]:
class PeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [60]:
train_dataset = PeDataset(inputs_train)

In [61]:
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=)
# maybe we don't need this.

### test dataset

In [62]:
df_test = df[df.split == 'TEST']

In [63]:
df_test = df_test.reset_index(drop=True)

In [64]:
df_test

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_1,prompted_rep_w_modif_MC
0,essay004,T1,MajorClaim 262 376,this industry has affected the cultural attrib...,MajorClaim,MajorClaim,[],,Linked,TEST,...,0,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, DET...",Topic: International tourism is now more commo...,Component: Topic: International tourism is now...,Component: Topic: International tourism is now...
1,essay004,T2,MajorClaim 1663 1758,the tourism has created threatening pressure o...,MajorClaim,MajorClaim,[],,Linked,TEST,...,0,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,1.00,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, VER...",Topic: International tourism is now more commo...,Component: Topic: International tourism is now...,Component: Topic: International tourism is now...
2,essay004,T3,Claim 179 239,the tourism bring large profit for the destina...,Claim,Claim,[],Attack,NotLinked,TEST,...,1,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, VERB, ADJ, NOU...",Topic: International tourism is now more commo...,Component: Topic: International tourism is now...,Component: Topic: International tourism is now...
3,essay004,T4,Claim 953 1031,international tourism can create negative impa...,Claim,Claim,[],Support,Linked,TEST,...,0,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.50,0,Topic: International tourism is now more commo...,"Part Of Speech tags: ADJ, NOUN, VERB, VERB, AD...",Topic: International tourism is now more commo...,Component: Topic: International tourism is now...,Component: Topic: International tourism is now...
4,essay004,T5,Claim 1578 1624,tourism has threatened the nature environments,Claim,Claim,[],Support,Linked,TEST,...,0,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.75,0,Topic: International tourism is now more commo...,"Part Of Speech tags: NOUN, AUX, VERB, DET, NOU...",Topic: International tourism is now more commo...,Component: Topic: International tourism is now...,Component: Topic: International tourism is now...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,essay398,T12,Claim 1484 1589,universities should encourage more girls to ch...,Claim,Claim,[],Support,Linked,TEST,...,2,Topic: We can not forcedly put the same number...,Topic: We can not forcedly put the same number...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: NOUN, VERB, VERB, ADJ, NO...",Topic: We can not forcedly put the same number...,Component: Topic: We can not forcedly put the ...,Component: Topic: We can not forcedly put the ...
1256,essay398,T13,Premise 1595 1648,this could avoid imbalance of gender in some s...,Premise,Premise,[],Support,NotLinked,TEST,...,1,Topic: We can not forcedly put the same number...,Topic: We can not forcedly put the same number...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: DET, VERB, VERB, NOUN, AD...",Topic: We can not forcedly put the same number...,Component: Topic: We can not forcedly put the ...,Component: Topic: We can not forcedly put the ...
1257,essay398,T14,Premise 1650 1734,It would affect students' mental health to stu...,Premise,Premise,[],Support,NotLinked,TEST,...,0,T

In [65]:
df_test.split.value_counts()

TEST    1260
Name: split, dtype: int64

In [66]:
prompted_texts_test = df_test['prompted_representation_1'][:].tolist()

In [67]:
len(prompted_texts_test)

1260

In [68]:
inputs_test = tokenizer(prompted_texts_test, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [69]:
inputs_test

{'input_ids': tensor([[ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        ...,
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0],
        [ 101, 6922, 1024,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [70]:
inputs_test['input_ids'].shape

torch.Size([1260, 512])

In [71]:
# for test we do not need labels

In [72]:
#inputs_train['labels'] = inputs_train.input_ids.detach().clone()

In [73]:
#inputs_train

In [74]:
#inputs_train['labels'].shape

In [75]:
# find where the last 102 is

In [76]:
#(inputs_train['input_ids'][0] == 102).nonzero(as_tuple=True)[0].item()

In [77]:
#tokenizer.decode(4366)

In [78]:
#tokenizer.decode(18458)

In [79]:
# claim = 4366, premise = 18458

In [80]:
#(inputs_train['input_ids'][1] == 102).nonzero(as_tuple=True)[0].item()

In [81]:
#inputs_train['input_ids'][1][169]

In [82]:
# confirm that all at [len-2] indices are either claim or premise

In [83]:
sep_indices_list_test = []

for i in range(len(prompted_texts_test)):
    sep_idx = (inputs_test['input_ids'][i] == 102).nonzero(as_tuple=True)[0].item()
    sep_indices_list_test.append(sep_idx)

In [84]:
sep_indices_list_test

[147,
 129,
 129,
 112,
 99,
 138,
 254,
 145,
 123,
 231,
 150,
 116,
 142,
 154,
 137,
 140,
 200,
 147,
 131,
 106,
 113,
 114,
 117,
 95,
 142,
 128,
 134,
 138,
 105,
 134,
 106,
 119,
 122,
 108,
 144,
 135,
 103,
 138,
 101,
 127,
 127,
 111,
 115,
 123,
 115,
 94,
 102,
 118,
 113,
 88,
 98,
 161,
 123,
 144,
 104,
 136,
 151,
 141,
 103,
 115,
 143,
 146,
 148,
 127,
 158,
 138,
 191,
 174,
 220,
 165,
 165,
 140,
 119,
 139,
 112,
 128,
 143,
 110,
 132,
 138,
 122,
 147,
 155,
 145,
 131,
 104,
 109,
 142,
 155,
 102,
 99,
 106,
 119,
 129,
 119,
 156,
 132,
 146,
 116,
 124,
 106,
 182,
 95,
 171,
 142,
 138,
 141,
 129,
 116,
 226,
 135,
 147,
 143,
 108,
 130,
 182,
 147,
 133,
 112,
 123,
 130,
 160,
 188,
 129,
 181,
 97,
 200,
 97,
 153,
 111,
 157,
 125,
 135,
 163,
 113,
 156,
 110,
 129,
 94,
 128,
 164,
 112,
 131,
 107,
 137,
 160,
 196,
 114,
 113,
 119,
 142,
 180,
 129,
 128,
 126,
 132,
 151,
 107,
 147,
 158,
 207,
 138,
 115,
 126,
 146,
 139,
 115,
 145,
 1

In [85]:
len(sep_indices_list_test)

1260

In [86]:
class_tokens_indices_test_list = [x - 2 for x in sep_indices_list_test]

In [87]:
class_tokens_indices_test_list

[145,
 127,
 127,
 110,
 97,
 136,
 252,
 143,
 121,
 229,
 148,
 114,
 140,
 152,
 135,
 138,
 198,
 145,
 129,
 104,
 111,
 112,
 115,
 93,
 140,
 126,
 132,
 136,
 103,
 132,
 104,
 117,
 120,
 106,
 142,
 133,
 101,
 136,
 99,
 125,
 125,
 109,
 113,
 121,
 113,
 92,
 100,
 116,
 111,
 86,
 96,
 159,
 121,
 142,
 102,
 134,
 149,
 139,
 101,
 113,
 141,
 144,
 146,
 125,
 156,
 136,
 189,
 172,
 218,
 163,
 163,
 138,
 117,
 137,
 110,
 126,
 141,
 108,
 130,
 136,
 120,
 145,
 153,
 143,
 129,
 102,
 107,
 140,
 153,
 100,
 97,
 104,
 117,
 127,
 117,
 154,
 130,
 144,
 114,
 122,
 104,
 180,
 93,
 169,
 140,
 136,
 139,
 127,
 114,
 224,
 133,
 145,
 141,
 106,
 128,
 180,
 145,
 131,
 110,
 121,
 128,
 158,
 186,
 127,
 179,
 95,
 198,
 95,
 151,
 109,
 155,
 123,
 133,
 161,
 111,
 154,
 108,
 127,
 92,
 126,
 162,
 110,
 129,
 105,
 135,
 158,
 194,
 112,
 111,
 117,
 140,
 178,
 127,
 126,
 124,
 130,
 149,
 105,
 145,
 156,
 205,
 136,
 113,
 124,
 144,
 137,
 113,
 143,
 13

In [88]:
# inputs_pe['input_ids'][0][143].item() # correct

In [89]:
# now get a list of input ids at all these indices: they should be either claim = 4366, premise = 18458

In [90]:
list_index_minus_2_test = []

for idx, val in enumerate(class_tokens_indices_test_list):
    at_idx_minus_2 = inputs_test['input_ids'][idx][val].item()
    list_index_minus_2_test.append(at_idx_minus_2)

In [91]:
set(list_index_minus_2)

{4366, 11032, 18458}

In [92]:
# correct

In [93]:
# now make the class token indices 103

In [94]:
for idx, val in enumerate(class_tokens_indices_test_list):
    inputs_test['input_ids'][idx][val] = 103

In [95]:
# now check if they are 103

In [96]:
unsontroi_check_list = []

for idx, val in enumerate(class_tokens_indices_test_list):
    at_idx_minus_2 = inputs_test['input_ids'][idx][val].item()
    unsontroi_check_list.append(at_idx_minus_2)

In [97]:
set(unsontroi_check_list)

{103}

In [98]:
# correct

In [99]:
# class PeDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings):
#         self.encodings = encodings
#     def __getitem__(self, idx):
#         return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#     def __len__(self):
#         return len(self.encodings.input_ids)

In [100]:
test_dataset = PeDataset(inputs_test)

In [132]:
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True)
# maybe we don't need this.

## get the model

In [101]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [102]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [103]:
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [104]:
# let's see with a simple trainer now and then we will try the fancy trainer.

In [105]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir = 'out',
    per_device_train_batch_size=16,
    num_train_epochs=2
)

In [106]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset
)

In [107]:
trainer.train()

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4713
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 590
/tmp/ipykernel_102/1625588428.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,0.158100


Saving model checkpoint to out/checkpoint-500
Configuration saved in out/checkpoint-500/config.json
Model weights saved in out/checkpoint-500/pytorch_model.bin
/tmp/ipykernel_102/1625588428.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=590, training_loss=0.1341243597915617, metrics={'train_runtime': 601.9405, 'train_samples_per_second': 15.659, 'train_steps_per_second': 0.98, 'total_flos': 2480968565452800.0, 'train_loss': 0.1341243597915617, 'epoch': 2.0})

In [108]:
trainer.save_model("pe_mask_pipeline_model")

Saving model checkpoint to pe_mask_pipeline_model
Configuration saved in pe_mask_pipeline_model/config.json
Model weights saved in pe_mask_pipeline_model/pytorch_model.bin


### Pipeline

In [109]:
saved_model = BertForMaskedLM.from_pretrained("/notebooks/Prompting/notebooks/pe_mask_pipeline_model")

loading configuration file /notebooks/Prompting/notebooks/pe_mask_pipeline_model/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.21.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /notebooks/Prompting/notebooks/pe_mask_pipeline_model/pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were initialize

In [23]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [110]:
from transformers import pipeline

In [112]:
classifier = pipeline("fill-mask", model=saved_model, tokenizer=tokenizer)

In [113]:
classifier.model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [114]:
targets = ['stance', 'claim', 'premise']

In [115]:
z = classifier(["Component: Topic: Should students be taught to compete or to cooperate?, Sentence: From this point of view, I firmly believe that we should attach more importance to cooperation during primary education., First or last in essay: Yes, First in paragraph: Yes, Last in paragraph: Yes, In in introduction: Yes, Is in conclusion: No. Part Of Speech tags: PRON, VERB, VERB, ADJ, NOUN, ADP, NOUN, ADP, ADJ, NOUN. Is this component a premise, a claim or a major claim? This component is a [MASK]."], targets=targets)

Disabling tokenizer parallelism, we're using DataLoader multithreading already


In [118]:
z

[{'score': 0.9651053547859192,
  'token': 11032,
  'token_str': 's t a n c e',
  'sequence': 'component : topic : should students be taught to compete or to cooperate?, sentence : from this point of view, i firmly believe that we should attach more importance to cooperation during primary education., first or last in essay : yes, first in paragraph : yes, last in paragraph : yes, in in introduction : yes, is in conclusion : no. part of speech tags : pron, verb, verb, adj, noun, adp, noun, adp, adj, noun. is this component a premise, a claim or a major claim? this component is a stance.'},
 {'score': 0.03204851597547531,
  'token': 4366,
  'token_str': 'c l a i m',
  'sequence': 'component : topic : should students be taught to compete or to cooperate?, sentence : from this point of view, i firmly believe that we should attach more importance to cooperation during primary education., first or last in essay : yes, first in paragraph : yes, last in paragraph : yes, in in introduction : ye

In [119]:
# ok now see how to do this for the whole list of test components

In [120]:
# but first we must replace the last token by [MASK]!

In [121]:
def get_masked_components(x):
    
    prompted_rep = x.prompted_rep_w_modif_MC
    
    last_token = prompted_rep.split()[-1]
    

    new_rep = prompted_rep.replace(last_token, "[MASK].")

    
    return new_rep

In [122]:
df_test

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_1,prompted_rep_w_modif_MC
0,essay004,T1,MajorClaim 262 376,this industry has affected the cultural attrib...,MajorClaim,MajorClaim,[],,Linked,TEST,...,0,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, DET...",Topic: International tourism is now more commo...,Component: Topic: International tourism is now...,Component: Topic: International tourism is now...
1,essay004,T2,MajorClaim 1663 1758,the tourism has created threatening pressure o...,MajorClaim,MajorClaim,[],,Linked,TEST,...,0,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,1.00,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, VER...",Topic: International tourism is now more commo...,Component: Topic: International tourism is now...,Component: Topic: International tourism is now...
2,essay004,T3,Claim 179 239,the tourism bring large profit for the destina...,Claim,Claim,[],Attack,NotLinked,TEST,...,1,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, VERB, ADJ, NOU...",Topic: International tourism is now more commo...,Component: Topic: International tourism is now...,Component: Topic: International tourism is now...
3,essay004,T4,Claim 953 1031,international tourism can create negative impa...,Claim,Claim,[],Support,Linked,TEST,...,0,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.50,0,Topic: International tourism is now more commo...,"Part Of Speech tags: ADJ, NOUN, VERB, VERB, AD...",Topic: International tourism is now more commo...,Component: Topic: International tourism is now...,Component: Topic: International tourism is now...
4,essay004,T5,Claim 1578 1624,tourism has threatened the nature environments,Claim,Claim,[],Support,Linked,TEST,...,0,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.75,0,Topic: International tourism is now more commo...,"Part Of Speech tags: NOUN, AUX, VERB, DET, NOU...",Topic: International tourism is now more commo...,Component: Topic: International tourism is now...,Component: Topic: International tourism is now...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,essay398,T12,Claim 1484 1589,universities should encourage more girls to ch...,Claim,Claim,[],Support,Linked,TEST,...,2,Topic: We can not forcedly put the same number...,Topic: We can not forcedly put the same number...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: NOUN, VERB, VERB, ADJ, NO...",Topic: We can not forcedly put the same number...,Component: Topic: We can not forcedly put the ...,Component: Topic: We can not forcedly put the ...
1256,essay398,T13,Premise 1595 1648,this could avoid imbalance of gender in some s...,Premise,Premise,[],Support,NotLinked,TEST,...,1,Topic: We can not forcedly put the same number...,Topic: We can not forcedly put the same number...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: DET, VERB, VERB, NOUN, AD...",Topic: We can not forcedly put the same number...,Component: Topic: We can not forcedly put the ...,Component: Topic: We can not forcedly put the ...
1257,essay398,T14,Premise 1650 1734,It would affect students' mental health to stu...,Premise,Premise,[],Support,NotLinked,TEST,...,0,T

In [123]:
df_test['masked_representation_for_pipeline'] = df_test.apply(lambda x: get_masked_components(x), axis=1)

In [124]:
df_test['masked_representation_for_pipeline'][0]

'Component: Topic: International tourism is now more common than ever before, Sentence: While some might think the tourism bring large profit for the destination countries, I would contend that this industry has affected the cultural attributes and damaged the natural environment of the tourist destinations., First or last in essay: Yes, First in paragraph: No, Last in paragraph: Yes, In in introduction: Yes, Is in conclusion: No. Part Of Speech tags: DET, NOUN, AUX, VERB, DET, ADJ, NOUN, CCONJ, VERB, DET, ADJ, NOUN, ADP, DET, NOUN, NOUN. Is this component a premise, a claim or a major claim? This component is a [MASK].'

In [125]:
# ok now do a sanity check to see that they are done correctly

In [128]:
last_tokens_mask_list = []

In [129]:
def get_last_token(x):
    
    prompted_rep = x.masked_representation_for_pipeline
    
    last_token = prompted_rep.split()[-1]
    
    last_tokens_mask_list.append(last_token)

In [131]:
df_test.apply(lambda x: get_last_token(x), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
1255    None
1256    None
1257    None
1258    None
1259    None
Length: 1260, dtype: object

In [132]:
last_tokens_mask_list

['[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[M

In [133]:
set(last_tokens_mask_list)

{'[MASK].'}

In [134]:
###### YASSSSS SIRRRRRRRRRRR

In [135]:
# ok now see how we can give all these 1260 statements to the pipeline

In [151]:
test_components_list = df_test['masked_representation_for_pipeline'][:].tolist()

In [152]:
len(test_components_list)

1260

In [153]:
# ok now give it to the pipeline

In [154]:
classifier_output = classifier(test_components_list, targets=targets)

In [156]:
classifier_output

[[{'score': 0.5339224934577942,
   'token': 11032,
   'token_str': 's t a n c e',
   'sequence': 'component : topic : international tourism is now more common than ever before, sentence : while some might think the tourism bring large profit for the destination countries, i would contend that this industry has affected the cultural attributes and damaged the natural environment of the tourist destinations., first or last in essay : yes, first in paragraph : no, last in paragraph : yes, in in introduction : yes, is in conclusion : no. part of speech tags : det, noun, aux, verb, det, adj, noun, cconj, verb, det, adj, noun, adp, det, noun, noun. is this component a premise, a claim or a major claim? this component is a stance.'},
  {'score': 0.4193692207336426,
   'token': 4366,
   'token_str': 'c l a i m',
   'sequence': 'component : topic : international tourism is now more common than ever before, sentence : while some might think the tourism bring large profit for the destination coun

In [167]:
classifier_output[0][0]['token']

11032

In [177]:
predicted_labels_test = []

for idx in range(1260):
    
    predicted_token = classifier_output[idx][0]['token']
    predicted_labels_test.append(predicted_token)

In [178]:
len(predicted_labels_test)

1260

In [179]:
# sanity check
set(predicted_labels_test)

{4366, 11032, 18458}

In [172]:
# ok correct

In [174]:
test_labels_str = df_test.label_ComponentType[:].tolist()

In [175]:
test_labels_str

['MajorClaim',
 'MajorClaim',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'MajorClaim',
 'MajorClaim',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Claim',
 'MajorClaim',
 'MajorClaim',
 'Claim',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'MajorClaim',
 'MajorClaim',
 'Claim',
 'Premise',
 'Premise',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'MajorClaim',
 'MajorClaim',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'MajorClaim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',

In [176]:
set(test_labels_str)

{'Claim', 'MajorClaim', 'Premise'}

In [180]:
test_labels_int = []

In [181]:
def get_int_labels():
    
    for idx, val in enumerate(test_labels_str):
        
        if val == 'MajorClaim':
            
            test_labels_int.append(11032)
            
        elif val == 'Claim':
            
            test_labels_int.append(4366)
            
        elif val == 'Premise':
            
            test_labels_int.append(18458)

In [182]:
get_int_labels()

In [183]:
test_labels_int

[11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 4366,
 11032,
 11032,
 4366,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 18458,
 18458,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 4366,
 11032,
 4366,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 11032,
 4366,
 11032,


In [184]:
len(test_labels_int)

1260

In [185]:
set(test_labels_int)

{4366, 11032, 18458}

In [186]:
# now do the classification report

In [187]:
from sklearn.metrics import classification_report

In [188]:
print(classification_report(test_labels_int, predicted_labels_test, digits=3))

              precision    recall  f1-score   support

        4366      0.661     0.536     0.592       302
       11032      0.756     0.869     0.809       153
       18458      0.881     0.918     0.899       805

    accuracy                          0.821      1260
   macro avg      0.766     0.775     0.767      1260
weighted avg      0.813     0.821     0.815      1260

